## Import all what we need

In [1]:
import os
import sys
import json
import requests
import pandas as pd
from io import StringIO
from tqdm import tqdm

/usr/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


# Change the Information Here

In [6]:
abspath = '/home/jingxin/'

# Read Master File

Read the master file 'ExoMol All'. The URL is http://www.exomol.com/db/exomol.all.

In [ ]:
exomol_all_url = 'http://www.exomol.com/db/exomol.all'

content = requests.get(exomol_all_url).text.replace('#','')
exomol_col_name = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6']
exomol_all = pd.read_csv(StringIO(content), sep='\\s+', names=exomol_col_name, header=None)

exomol_all

Get all molecules, their iso-slugs, isoFormula and isotopologue dataset names. 

In [ ]:
first = exomol_all['c1']
fourth = exomol_all['c4']
row = len(first)

iso_slug = pd.DataFrame()
iso_formula = pd.DataFrame()
iso_dataset = pd.DataFrame()
molecule_unduplicated = pd.DataFrame()
num_iso_in_mol = pd.DataFrame()

for i in tqdm(range(row)):
        
    _iso_slug = exomol_all[first.isin(['Iso-slug'])]['c0'].values
    _iso_formula = exomol_all[first.isin(['IsoFormula'])]['c0'].values
    _iso_dataset = exomol_all[first.isin(['Isotopologue'])]['c0'].values
    _molecule_unduplicated = exomol_all[first.isin(['Molecule'])]['c0'].values
    _num_iso_in_mol = exomol_all[fourth.isin(['considered'])]['c0'].values

iso_slug = iso_slug.append(pd.DataFrame(_iso_slug))
iso_formula = iso_formula.append(pd.DataFrame(_iso_formula))
iso_dataset = iso_dataset.append(pd.DataFrame(_iso_dataset))
molecule_unduplicated = molecule_unduplicated.append(pd.DataFrame(_molecule_unduplicated))
num_iso_in_mol = num_iso_in_mol.append(pd.DataFrame(_num_iso_in_mol))
isotopologue_unduplicated = iso_slug.drop_duplicates(subset=None, keep='first', inplace=False)
iso_datset_unduplicated = iso_dataset.drop_duplicates(subset=None, keep='first', inplace=False)

100%|██████████| 1500/1500 [00:02<00:00, 699.15it/s]


In [ ]:
iso_slug

,0
0,1H2-16O
1,1H2-17O
2,1H2-18O
3,1H-2H-16O
4,1H-2H-16O
...,...
218,trans-31P2-1H2
219,1H3-16O_p
220,39K-16O-1H
221,23Na-16O-1H


In [ ]:
iso_formula

,0
0,(1H)2(16O)
1,(1H)2(17O)
2,(1H)2(18O)
3,(1H)(2H)(16O)
4,(1H)(2H)(16O)
...,...
218,trans-(31P)2(1H)2
219,(1H)3(16O)+
220,(39K)(16O)(1H)
221,(23Na)(16O)(1H)


In [ ]:
iso_dataset

,0
0,POKAZATEL
1,HotWat78
2,HotWat78
3,VTT
4,Hewitt
...,...
218,OY-Trans
219,eXeL
220,OYT4
221,OYT5


All molecule and their corresponding isotopologue dataset numbers.

In [ ]:
molecule_unduplicated

,0
0,H2O
1,CO2
2,CO
3,CH4
4,NO
...,...
76,trans-P2H2
77,H3O+
78,KOH
79,NaOH


In [ ]:
num_iso_in_mol

,0
0,6
1,14
2,6
3,2
4,7
...,...
76,2
77,1
78,1
79,1


In [ ]:
isotopologue_unduplicated

,0
0,1H2-16O
1,1H2-17O
2,1H2-18O
3,1H-2H-16O
5,2H2-16O
...,...
218,trans-31P2-1H2
219,1H3-16O_p
220,39K-16O-1H
221,23Na-16O-1H


In [ ]:
iso_datset_unduplicated

,0
0,POKAZATEL
1,HotWat78
3,VTT
4,Hewitt
6,Zak-16-12-16
...,...
217,OY-Trans
219,eXeL
220,OYT4
221,OYT5


Set the length of molecules list to be the same as the length of iso-slugs and isotopologue dataset names for following loop.

In [ ]:
molecule_repeated = pd.DataFrame()
molecule_num = len(molecule_unduplicated)

for j in range(molecule_num):    
    molecule_repeated = molecule_repeated.append(pd.DataFrame((molecule_unduplicated.values[j] + ' ')
                                                              * int(num_iso_in_mol.values[j])))

molecule_str = (str(molecule_repeated.values).replace("[['"," ")
                .replace("']\n ['"," ").replace("']]"," ").replace("+","_p"))
molecule = pd.read_csv(StringIO(molecule_str), sep='\s+', header=None)
molecule


,0,1,2,3,4,5,6,7,8,9,...,213,214,215,216,217,218,219,220,221,222
0,H2O,H2O,H2O,H2O,H2O,H2O,CO2,CO2,CO2,CO2,...,CH3,SiH2,SiO2,cis-P2H2,trans-P2H2,trans-P2H2,H3O_p,KOH,NaOH,NaO


In [ ]:
molecule.values

array([['H2O', 'H2O', 'H2O', 'H2O', 'H2O', 'H2O', 'CO2', 'CO2', 'CO2',
        'CO2', 'CO2', 'CO2', 'CO2', 'CO2', 'CO2', 'CO2', 'CO2', 'CO2',
        'CO2', 'CO2', 'CO', 'CO', 'CO', 'CO', 'CO', 'CO', 'CH4', 'CH4',
        'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'NO', 'SO2', 'NH3', 'NH3',
        'NH3', 'NH3', 'HNO3', 'OH', 'HF', 'HCl', 'HCl', 'HCl', 'HCl',
        'H2CO', 'N2', 'HCN', 'HCN', 'CH3Cl', 'CH3Cl', 'H2O2', 'H2O2',
        'C2H2', 'PH3', 'PH3', 'H2S', 'C2H4', 'CS', 'CS', 'CS', 'CS',
        'CS', 'CS', 'CS', 'CS', 'H2', 'H2', 'H2', 'LiH', 'LiH_p', 'FeH',
        'TiH', 'BeH', 'BeH', 'BeH', 'BeH', 'CaH', 'CaH', 'CrH', 'SiH',
        'SiH', 'SiH', 'SiH', 'AlH', 'AlH', 'AlH', 'NaH', 'NaH', 'MgH',
        'MgH', 'MgH', 'MgH', 'SiO', 'SiO', 'SiO', 'SiO', 'SiO', 'SiO',
        'SiO', 'TiO', 'TiO', 'TiO', 'TiO', 'TiO', 'MgO', 'MgO', 'MgO',
        'MgO', 'MgO', 'H3_p', 'H3_p', 'H3_p', 'HeH_p', 'HeH_p', 'HeH_p',
        'HeH_p', 'HeH_p', 'HeH_p', 'HeH_p', 'HeH_p', 'H2_p', 'H2_p',
       

# Read Def File

Get all URLs of def files. The number of def files should be the same as the number of isotopologue datasets. The URLs contains the names of molecules, iso-slugs and isotopologue datasets. We save their corresponding isoFormula names as another column.

In [ ]:
def_url = pd.DataFrame()

def_num = len(iso_slug)

for i in tqdm(range(def_num)):
    def_url = def_url.append('http://www.exomol.com/db/' + molecule[i] + '/'
                             + iso_slug.values[i] + '/'+ iso_dataset.values[i] + '/'
                             + iso_slug.values[i] + '__' + iso_dataset.values[i] + '.def')

    
def_url['IsoFormula'] = iso_formula
def_url.columns = ['def url','IsoFormula']

def_url

In [ ]:
def_url.values[0:10]

Download def files and save them into ./data/def/ folder. Save the names of these def files with all information we got before, that is to say, save as 'molecule_isoFormula_iso-slug_isotopologue.def'. It will be more convenient for processing data later.

In [ ]:
def download_deffile(file_url):
    
    failed_list = [] 
    for _link in tqdm(file_url['def url'].values):
        
        link = _link
        iso_slug = link.split('/')[-4]
        iso_formula = (str(file_url[file_url['def url'].isin([link])]['IsoFormula'].values)
                       .replace("['","").replace("']",""))
        inital_def_name = link.split('/')[-1]
        new_def_filename = iso_slug + '_' + iso_formula + '_' + inital_def_name
        print("Downloading file: %s" % new_def_filename)
        print(link)
 
        # Make folders for save doanloaded files.
        folder_name = abspath + '/data/def/'
        if os.path.exists(folder_name):
            pass
        else:
            os.makedirs(folder_name, exist_ok=True)
        filename = os.path.join(folder_name, new_def_filename)
        
        try:
            r = requests.get(link, stream=True, verify=False)
        except Exception:
            failed_list.append(new_def_filename)
            print(' download failed. Go to download next one\n')
              
        # For compute the progess.
        total_size = int(r.headers['Content-Length'])
        temp_size = 0    
   
        # Download started.
        with open(filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    temp_size += len(chunk)
                    f.write(chunk)
                    f.flush()
                    done = int(50 * temp_size / total_size)
                    sys.stdout.write("\r[%s%s] %d%%" % ('█' * done, ' ' * (50 - done),
                                                        100 * temp_size / total_size))
                    sys.stdout.flush()
        
        print(" Downloaded!\n")

    print("All def files downloaded!\n")    
    print("The files which are failed to download: \n")
    print(failed_list) # Record which file is failed to download.

    return


In [ ]:
download_deffile(def_url)

Extract the def files whose the uncertainty availability row shows YES.

In [ ]:
path = abspath + '/data/def'

def_col_name = ['c0', '#', 'c1', 'c2', 'c3', 'c4', 'c5']

for(dirpath,dirnames,files)in os.walk(path):
    tot = 0
    count = 0
    unc_def_filename = []
    for filename in files:
        filepath = os.path.join(dirpath, filename)
        tot += 1
        def_df =  pd.read_csv(filepath,sep='\s+', names=def_col_name, header=None)
        c1 = def_df['c1']
        if def_df[c1.isin(['Uncertainty'])]['c0'].values == '1':
            unc_def_filename.append(filename)
            count += 1            
        else:
            no_unc_def_filename = []
        
    print('There are ', tot, ' def files.\n')
    print('The uncertainty availability shows YES in the following ', count, ' def files:\n', unc_def_filename)
    print('\nThe uncertainty availability does not exit or shows NO in other ', tot - count, 'def files.')
                    

In [ ]:
unc_def_filename

# Get Download Links with API

Get the API URLs of those uncertainty molecules.

In [ ]:
molecule_str = []
iso_formula_str = []
iso_dataset_str = []
iso = []
api_url = []
target_link = []

for i in range(len(unc_def_filename)):
    molecule_str.append(unc_def_filename[i].replace('_p','+').split('_')[0].replace('+','_p'))
    iso_formula_str.append(unc_def_filename[i].replace('_p','+').split('_')[1])
    iso_dataset_str.append(unc_def_filename[i].split('_')[-1].split('.')[0])
    
    _iso = (iso_formula_str[i], iso_dataset_str[i])
    iso.append(_iso)
    
    api_url.append('http://exomol.com/api/?molecule=*&datatype=linelist'.replace('*',molecule_str[i]))
    
print(iso)
api_url

Get the download links of states.bz2 files and trans.bz2 files from API.

In [ ]:
def get_target_url():
    
    """
    Get the download url from API.

    """
    
    file_url = []
    for i in range(len(iso)):
        
        response = requests.get(api_url[i])

        if(response.status_code != 200):
            print('ExoMol API Error' + str(response.status_code))

        # If the obtained status code is 200, it is correct.
        else:
            content = response.text            # Get the relevant content.
            json_dict = json.loads(content)    # Convert json into dictionary.

            # Extract files information from dictionary and convert them into list
            iso_slug = iso[i][0]
            iso_dataset = iso[i][1]
            json_list = json_dict[iso_slug]['linelist'][iso_dataset]['files']

            print('The number of downloading files for', iso_slug, iso_dataset, ': ', len(json_list))
            print("Download links:")
            
            url_show = []
            for j in range(len(json_list)):
                link = json_list[j].get('url')
                try:
                    if((link.endswith('states.bz2') or link.endswith('trans.bz2'))):
                        file_url.append("http://www." + link)
                        url_show.append("http://www." + link)
                except KeyError:
                    print('Keyerror, keep going!')
                    
    
        for k in url_show:
            print(k)

    return file_url


In [ ]:
target_link = get_target_url()

# Download States and Trans Files

We write all the download URLs into a text file, name it as $api__urls.txt$. 

In Linux, we use command 

```
wget -d -r -i /.../save_path/.../api__urls.txt
```

Download states.bz2 files and trans.bz2 files with download links. Save these files into correspoding folders.


In [ ]:
url_path = abspath + '/data/url'
url_filename = url_path + '/api__urls.txt'

if os.path.exists(url_path):
    pass
else:
    os.makedirs(url_path, exist_ok=True)

with open(url_filename, 'w') as file:
    file.write('\n'.join(target_link))